# Raso lark demographic analysis using dadi

In [ ]:
# Numpy is the numerical library dadi is built upon
import numpy as np
from numpy import array

import pylab

import dadi

# Imort the models that we have written for this scenario
import dadi_models

import matplotlib.pyplot as plt

In [ ]:
#NOTE there are different optimisaition methods here I use dadi.Inference.optimize_log because dadi.Inference.optimize seems to get stuck more easily.
def runOptimisation(func_ex,pts_l,data,params,upper_bound,lower_bound,fixed_params=None,maxiter=10,peturbFold=1):
    
    if fixed_params is None: fixed_params = [None]*len(params)
    
    # Perturb our parameter array before optimization.
    p0 = dadi.Misc.perturb_params(params, fold=peturbFold, upper_bound=upper_bound, lower_bound=lower_bound)
    
    popt = dadi.Inference.optimize_log(p0, data, func_ex, pts_l, 
                                    lower_bound=lower_bound,
                                    upper_bound=upper_bound,
                                    verbose=len(params),
                                    maxiter=maxiter,
                                    fixed_params = fixed_params)
    
    return popt

def get_fitted_model(func_ex,pts_l,data,params):
    output = {}
    model = func_ex(params, data.sample_sizes, pts_l)
    output['params'] = params
    output['model'] = model
    output['theta'] = dadi.Inference.optimal_sfs_scaling(model, data)
    output['ll_opt'] = dadi.Inference.ll_multinom(model, data)
    
    return output



In [ ]:
#function to convert generic sfs to dadi formatted text file
def fsToDadiFormat(arr):
    return str(len(arr)) + "\n" + " ".join([str(i) for i in arr])

In [ ]:
def plotSpectrumBar(fs, width = None, col="black"):
    fs = np.array(fs)
    #if 1D, make 2D array
    shape = fs.shape
    if len(shape)==1: fs = fs.reshape([1,shape[0]])
    x = np.arange(fs.shape[1])
    if width is None: width = 1./fs.shape[0]
    col = list(col) * fs.shape[0]
    for i in range(fs.shape[0]): plt.bar(x + width*i, fs[i,:], width, color = col[i])
    plt.xlabel="Frequency"
    plt.show()

In [ ]:
def fold(arr):
    mid = len(arr)/2
    firstHalf = arr[:mid]
    secondHalf = arr[mid:][::-1]
    if len(secondHalf) > mid: firstHalf = np.append(firstHalf, 0)
    return firstHalf + secondHalf

In [ ]:
def extractSpec1D(spec2D, axis = 0):
    specCopy = spec2D.copy()
    specCopy[specCopy.mask] = 0.0
    return np.apply_along_axis(np.sum,axis,specCopy)

In [ ]:
def writeModel(model,fileName):
    with open(fileName,"w") as m:
        for i in model.items():
            m.write(i[0] + "\n")
            try: m.write("\t".join([str(j) for j in i[1]]) + "\n\n")
            except: m.write(str(i[1]) + "\n\n")

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10,6]

In [ ]:
mu = 2.3e-9
gen_time_raso = 6.5
gen_time_skylark = 1
gen_time_average = 3

## Getting the sfs

These SFSs are gnerated using [realSFS in ANGSD](http://www.popgen.dk/angsd/index.php/SFS_Estimation)

20 bootstraps were run. First step is to import these and take the median.
These include invariant sites in the first and last category, so the sum is the total number of sites used, which is needed later to scale parameters.

In [ ]:
file_raso = 'dadi/raso.NRscafs.baq1MQ1Q20GL2.BS20.sfs'
file_sky = 'dadi/skylarkNL.NRscafs.baq1MQ1Q20GL2.BS20.sfs'

fsArray_raso = np.loadtxt(file_raso)
fsArray_sky = np.loadtxt(file_sky)

#take the median as the one to use
fs_raso = np.median(fsArray_raso, axis = 0)
fs_sky = np.median(fsArray_sky, axis = 0)

#total number of sites is necessary for calculating scaling parameters
L_raso = np.sum(fs_raso)
L_sky = np.sum(fs_sky)

Easiest way to load SFS in dadi is from a file. So write to a file and reload in dadi.

In [ ]:
with open(file_raso.rstrip(".sfs") + ".dadi.sfs", "w") as ff: ff.write(fsToDadiFormat(fs_raso))
with open(file_sky.rstrip(".sfs") + ".dadi.sfs", "w") as ff: ff.write(fsToDadiFormat(fs_sky))

In [ ]:
#load sfs as dadi object
data_raso = dadi.Spectrum.from_file(file_raso.rstrip(".sfs") + ".dadi.sfs")
data_sky = dadi.Spectrum.from_file(file_sky.rstrip(".sfs") + ".dadi.sfs")


In [ ]:
data_raso.sample_sizes

In [ ]:
data_sky.sample_sizes

SFS is unfolded, but polarised with referecne, so it must be folded.

In [ ]:
data_raso = data_raso.fold()
data_sky = data_sky.fold()

Plot sfs for each population

In [ ]:
plt.rcParams['figure.figsize'] = [10,12]
plt.subplot(2,1,1)
plotSpectrumBar(data_raso[~data_raso.mask], width=0.8, col = ["black"])
plt.subplot(2,1,2)
plotSpectrumBar(data_sky[~data_sky.mask], width=0.8, col = ["black"])

### Nucleotide diversity ($\pi$) and Tajima's D###
We can estimate ${\pi}$ and tajima's D from the SFS

In [ ]:
print "pi Raso:", round(data_raso.pi()/L_raso,4)
print "pi Skylark:", round(data_sky.pi()/L_sky,4)

print "Tajima's D Raso:", data_raso.Tajima_D()
print "Tajima's D Skylark:", data_sky.Tajima_D()


## Single Population Models
The model used allows up to two changes in *Ne*. For constant and single change models, some of the parameters are fixed.

In [ ]:
# These are the grid point settings will use for extrapolation.
pts_l = [50,60,70]

In [ ]:
#model function
func = dadi_models.onePop_twoChange
func_ex = dadi.Numerics.make_extrap_func(func)
#func_ex = dadi.Numerics.make_extrap_log_func(func)

N1 = 1
N2 = 1
T1 = 1
T2 = 1
params = array([N1, N2, T1, T2])
upper_bound = [100, 100, 5, 5]
lower_bound = [1e-5, 1e-5,1e-5,1e-5]

## Constant population size

In [ ]:
constant_raso = get_fitted_model(func_ex,pts_l,data_raso,params=[1,1,1,1])
constant_sky = get_fitted_model(func_ex,pts_l,data_sky,params=[1,1,1,1])


In [ ]:
Nanc = constant_raso["theta"] / (4 * mu * L_raso)
print "Likelihood:", constant_raso["ll_opt"]
print "N:", Nanc
model_folded = constant_raso["model"].fold()
plt.rcParams['figure.figsize'] = [8,6]

fig = pylab.figure(1)
fig.clear()
plotSpectrumBar([data_raso[~data_raso.mask],model_folded[~model_folded.mask]*constant_raso["theta"]], width=0.4, col = ["black","red"])
fig.savefig("dadi/constant.pdf", format="pdf", figsize=(3,2.5))
writeModel(constant_raso, "dadi/fitted_constant_raso.txt")

In [ ]:
Nanc = constant_sky["theta"] / (4 * mu * L_sky)
print "Likelihood:", constant_sky["ll_opt"]
print "N:", Nanc
model_folded = constant_sky["model"].fold()
plt.rcParams['figure.figsize'] = [8,6]
plotSpectrumBar([data_sky[~data_sky.mask],model_folded[~model_folded.mask]*constant_sky["theta"]], width=0.4, col = ["black","red"])

## Single change in population size
There is a constant ancestral population (*N<sub>anc</sub>*), and the population changes to size *N<sub>1</sub>* *T<sub>1</sub>* years ago (times are estimated in generations and converted to years by multiplying by average generation time). 

In [ ]:
popt_oneChange_raso_all = [runOptimisation(func_ex,pts_l,data_raso,params,upper_bound,lower_bound, fixed_params=[1,None,1,None]) for i in range(10)]
popt_oneChange_sky_all = [runOptimisation(func_ex,pts_l,data_sky,params,upper_bound,lower_bound, fixed_params=[1,None,1,None]) for i in range(10)]

oneChange_raso_all = [get_fitted_model(func_ex,pts_l,data_raso,params=p) for p in popt_oneChange_raso_all]
oneChange_sky_all = [get_fitted_model(func_ex,pts_l,data_sky,params=p) for p in popt_oneChange_sky_all]

ll_opt_oneChange_raso = [result["ll_opt"] for result in oneChange_raso_all]
ll_opt_oneChange_sky = [result["ll_opt"] for result in oneChange_sky_all]

oneChange_raso = oneChange_raso_all[ll_opt_oneChange_raso.index(max(ll_opt_oneChange_raso))]
oneChange_sky = oneChange_sky_all[ll_opt_oneChange_sky.index(max(ll_opt_oneChange_sky))]

In [ ]:
plt.plot(ll_opt_oneChange_raso)
plt.show()

In [ ]:
model = oneChange_raso 
Nanc = model["theta"] / (4 * mu * L_raso)
print "Likelihood:", model["ll_opt"]
print "Nanc:", Nanc
print "N1:", Nanc*model["params"][1]
print "T1:", model["params"][3] * 2*Nanc * gen_time_raso

model_folded = model["model"].fold()
model_folded
fig = pylab.figure(1)
fig.clear()
plotSpectrumBar([data_raso[~data_raso.mask],model_folded[~model_folded.mask]*model["theta"]], width=0.4, col = ["black","red"])
fig.savefig("dadi/one_change.pdf", format="pdf", figsize=(3,2.5))
writeModel(oneChange_raso, "dadi/fitted_oneChange_raso.txt")

In [ ]:
Nanc = oneChange_sky["theta"] / (4 * mu * L_sky)
print "Likelihood:", oneChange_sky["ll_opt"]
print "Nanc:", Nanc
print "N1:", Nanc*oneChange_sky["params"][1]
print "T1:", oneChange_sky["params"][1] * 2*Nanc * gen_time_skylark

model_folded = oneChange_sky["model"].fold()
plotSpectrumBar([data_sky[~data_sky.mask],model_folded[~model_folded.mask]*oneChange_sky["theta"]], width=0.4, col = ["black","red"])


## Two changes in population size
There is a constant ancestral population (*N<sub>anc</sub>*), and the population changes to size *N<sub>1</sub>*, which it maintains for *T<sub>1</sub>* years, and then changes to *N<sub>2</sub>*, which is maintained for *T<sub>2</sub>* years, until the present (times are estimated in generations and converted to years by multiplying by average generation time). 

In [ ]:
#model function
func = dadi_models.onePop_twoChange
func_ex = dadi.Numerics.make_extrap_func(func)
#func_ex = dadi.Numerics.make_extrap_log_func(func)

N1 = .5
N2 = 1.
T1 = .5
T2 = .5
params = array([N1, N2, T1, T2])
upper_bound = [10, 10, 2, 2]
lower_bound = [1e-5, 1e-5,1e-5,1e-5]

In [ ]:
popt_twoChange_raso_all = [runOptimisation(func_ex,pts_l,data_raso,params,upper_bound,lower_bound, fixed_params=[None,None,None,None]) for i in range(50)]
popt_twoChange_sky_all = [runOptimisation(func_ex,pts_l,data_sky,params,upper_bound,lower_bound, fixed_params=[None,None,None,None]) for i in range(50)]

twoChange_raso_all = [get_fitted_model(func_ex,pts_l,data_raso,params=p) for p in popt_twoChange_raso_all]
twoChange_sky_all = [get_fitted_model(func_ex,pts_l,data_sky,params=p) for p in popt_twoChange_sky_all]

ll_opt_twoChange_raso = [result["ll_opt"] for result in twoChange_raso_all]
ll_opt_twoChange_sky = [result["ll_opt"] for result in twoChange_sky_all]

twoChange_raso = twoChange_raso_all[ll_opt_twoChange_raso.index(max(ll_opt_twoChange_raso))]
twoChange_sky = twoChange_sky_all[ll_opt_twoChange_sky.index(max(ll_opt_twoChange_sky))]

In [ ]:
plt.rcParams['figure.figsize'] = [10,12]
plt.subplot(2,1,1)
plt.plot(ll_opt_twoChange_raso)
plt.subplot(2,1,2)
plt.plot(ll_opt_twoChange_sky)

In [ ]:
model = twoChange_raso 
Nanc = model["theta"] / (4 * mu * L_raso)
print "Likelihood:", model["ll_opt"]
print "Raw estimates:", model["params"]
print "Nanc:", Nanc
print "N1:", Nanc*model["params"][0]
print "N2:", Nanc*model["params"][1]
print "T1:", model["params"][2] * 2*Nanc * gen_time_raso
print "T2:", model["params"][3] * 2*Nanc * gen_time_raso

model_folded = model["model"].fold()
model_folded
fig = pylab.figure(1)
fig.clear()
plt.rcParams['figure.figsize'] = [10,6]
plotSpectrumBar([data_raso[~data_raso.mask],model_folded[~model_folded.mask]*model["theta"]], width=0.4, col = ["black","red"])
fig.savefig("dadi/two_change.pdf", format="pdf", figsize=(3,2.5))
writeModel(twoChange_raso, "dadi/fitted_twoChange_raso.txt")

In [ ]:
Nanc = twoChange_sky["theta"] / (4 * mu * L_sky)
print "Likelihood:", twoChange_sky["ll_opt"]
print "Nanc:", Nanc
print "N1:", Nanc*twoChange_sky["params"][0]
print "N2:", Nanc*twoChange_sky["params"][1]
print "T1:", twoChange_sky["params"][2] * 2*Nanc * gen_time_skylark
print "T2:", twoChange_sky["params"][3] * 2*Nanc * gen_time_skylark

model_folded = twoChange_sky["model"].fold()
plt.rcParams['figure.figsize'] = [10,6]
plotSpectrumBar([data_sky[~data_sky.mask],model_folded[~model_folded.mask]*twoChange_sky["theta"]], width=0.4, col = ["black","red"])


## Three changes in population size
There is a constant ancestral population (*N<sub>anc</sub>*), and the population changes to size *N<sub>1</sub>*, which it maintains for *T<sub>1</sub>* years, and then changes to *N<sub>2</sub>*, which is maintained for *T<sub>2</sub>* years, and then changes to *N<sub>3</sub>*, which is maintained for *T<sub>3</sub>* years, until the present (times are estimated in generations and converted to years by multiplying by average generation time). 

In [ ]:
#model function
func = dadi_models.onePop_threeChange
func_ex = dadi.Numerics.make_extrap_func(func)
#func_ex = dadi.Numerics.make_extrap_log_func(func)

N1 = 1.
N2 = 1.
N3 = .01

T1 = .5
T2 = .5
T3 = .0001
params = array([N1, N2, N3, T1, T2, T3])
upper_bound = [100, 100, 20, 2, 2, 1]
lower_bound = [1e-3, 1e-3, 1e-4, 1e-5, 1e-5, 1e-5]

In [ ]:
N3range = np.arange(1e-3,1e-2,5e-4).repeat(50)
N3range = np.array([1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 5, 10, 50]).repeat(10)

popt_threeChange_raso_all = [runOptimisation(func_ex,pts_l,data_raso,params,upper_bound,lower_bound, maxiter=100,
                                             fixed_params=[None,None,N3i,None,None,None]) for N3i in N3range]

threeChange_raso_all = [get_fitted_model(func_ex,pts_l,data_raso,params=p) for p in popt_threeChange_raso_all]

ll_opt_threeChange_raso = [result["ll_opt"] for result in threeChange_raso_all]

threeChange_raso = threeChange_raso_all[ll_opt_threeChange_raso.index(max(ll_opt_threeChange_raso))]


In [ ]:
plt.rcParams['figure.figsize'] = [10,6]
plt.plot(ll_opt_threeChange_raso)


In [ ]:
model = threeChange_raso 
Nanc = model["theta"] / (4 * mu * L_raso)
print "Likelihood:", model["ll_opt"]
print "Raw estimates:", model["params"]
print "Nanc:", Nanc
print "N1:", Nanc*model["params"][0]
print "N2:", Nanc*model["params"][1]
print "N3:", Nanc*model["params"][2]
print "T1:", model["params"][3] * 2*Nanc * gen_time_raso
print "T2:", model["params"][4] * 2*Nanc * gen_time_raso
print "T3:", model["params"][5] * 2*Nanc * gen_time_raso

model_folded = model["model"].fold()
model_folded
fig = pylab.figure(1)
fig.clear()
plt.rcParams['figure.figsize'] = [10,6]
plotSpectrumBar([data_raso[~data_raso.mask],model_folded[~model_folded.mask]*model["theta"]], width=0.4, col = ["black","red"])
fig.savefig("dadi/three_change.pdf", format="pdf", figsize=(3,2.5))
writeModel(threeChange_raso, "dadi/fitted_threeChange_raso.txt")